# Making Asynchronous requests

### Introduction

Now so far we have seen how to create multiple asynchronous tasks with Python.  However in our examples of this we have only used the sleep function.  A common application of using asynchronous programming is in fetching data from an API.  

### An example

For example, let's say we want to make a separate request about each song by Billy Eilish.  If we do this synchronously, we'll have to wait for a single request to be finished before making the next request.  So if it takes two seconds to make each request, then with our standard synchronous style, fetching five different songs will take about ten seconds.  

However, with an asynchronous style we can fire off five requests in a row. Then, because we were able to issue these requests immediately, we'll get each of the responses about two seconds later.  

Ok, now let's see how we can issue an asynchronous request.  We'll do so using the aiohttp library.

### Implementing Async Requests

> If you look at the `async_requests.py` file, you'll find code that looks like the following.

> Do not try to execute this code in a Jupyter notebook, it won't work.

```python
import aiohttp
import asyncio
import json

async def fetch(url, params = {}):
    async with aiohttp.ClientSession() as session:
        async with session.get(url, params = params) as response:
            return await response.text()

async def get_artist(artist_name):
    url = "https://itunes.apple.com/search"
    
    params = {
        "term": artist_name,
        "entity": "album",
        "limit": 5,
        "sort": "recent",
    }
    response_text = await fetch(url, params = params)
    response_json = json.loads(response_text)
    matching_result = [result for result in response_json['results'] if result['artistName'] == artist_name]
    print(matching_result)

def get_artists(artist_names):
    return [get_artist(artist_name) for artist_name in artist_names]

async def main(artist_names):
    coros = get_artists(artist_names)
    await asyncio.gather(*coros)


artist_names = ['Billy Eilish', 'Dua Lipa', 'Lorde']
asyncio.run(main(artist_names))
```

Let's walk through this code in turn.

### Code walk through

* The fetch function

```python
import aiohttp
import asyncio
import json

async def fetch(url, params = {}):
    async with aiohttp.ClientSession() as session:
        async with session.get(url, params = params) as response:
            return await response.text()
```

With the fetch function, we are using our new `aiohttp` library to generate a clientsession object.  And then this object has a get method that makes our request.  We call `async with session.get` so that we can fire off multiple get requests at once, and have each be registered by the event loop.  Each of our coroutines needs in a call to await, and ours is at the end of the function.

> The above is pretty boiler plate code, and you can copy and paste it to generate a fetch function.  But for a more detailed explanation, see the notes below.

* Get artist

```python
async def get_artist(artist_name):
    url = "https://itunes.apple.com/search"
    
    params = {
        "term": artist_name,
        "entity": "album",
        "limit": 5,
        "sort": "recent",
    }
    response_text = await fetch(url, params = params)
    response_json = json.loads(response_text)
```

In this function we define a coroutine that makes a request to an individual artist.  We use  `await fetch` as we want each coroutine to wait until receiving the response text.  The fetch function returns text, so we then have to turn it into json.

The rest of the `get_artist` function looks like the following.

```python
matching_result = [result for result in response_json['results'] if result['artistName'] == artist_name]
print(matching_result)
```

Here, because the api returns a list of dictionaries of our potential artist, and some incorrect matches, filter through the dictionaries looking for the one that matches our artistname.

* Finishing up

```python
def get_artists(artist_names):
    return [get_artist(artist_name) for artist_name in artist_names]

async def main(artist_names):
    coros = get_artists(artist_names)
    await asyncio.gather(*coros)


artist_names = ['Billy Eilish', 'Dua Lipa', 'Lorde']
asyncio.run(main(artist_names))
```

Starting from the bottom, the `run` function must take a coroutine, so ours is the `main` function.  Then the `asyncio.gather` function takes a list of coroutines that it can execute one after the other, before waiting for any individual coroutine to complete.  

Notice that the `get_artists` function is not defined with async.  This is because we want it to immediately generate a list of coroutines (by calling get_artist multiple times), and them these are the coroutines passed to the `asyncio.gather` function.

### Seeing it in action

Ok, so now let's see the benefits of implementing our code this way.  If you call the `normal_requests.py` file, you'll see that these five requests can take a few seconds (between 2 and 5 seconds).  

`python3 normal_requests.py`

But calling `async_requests.py` should take less than a second.

`python3 normal_requests.py`

So we can see that even with just a few requests, we are able to save a significant amount of time.

### Summary

In this lesson, we saw how to make asynchronous requests with Python.  We did so using the `aiohttp` library, defining the `fetch` function that makes asynchronous requests with some boilerplate code.

Then, we defined a coroutine in `get_artist` that used the fetch function to call our api.

```python
async def get_artist(artist_name):
    url = "https://itunes.apple.com/search" 
    params = {
        "term": artist_name,
        "entity": "album",
    }
    response_text = await fetch(url, params = params)
    response_json = json.loads(response_text)
```

Finally, we generated a list of these coroutines, one for each artist name, and passed these coroutines into the `gather` function so that they can be executed immediately.

```python
def get_artists(artist_names):
    return [get_artist(artist_name) for artist_name in artist_names]

async def main(artist_names):
    coros = get_artists(artist_names)
    await asyncio.gather(*coros)
```

### Notes
```python
async with aiohttp.ClientSession() as session:
        async with session.get(url, params = params) as response:
            return await response.text()
```

This code snippet is a simple example of using the aiohttp library in Python to make an asynchronous GET request to a given URL and return the response text. The aiohttp library allows you to make asynchronous HTTP requests using Python's asyncio library. Here's a step-by-step explanation of the code:
```python
async with aiohttp.ClientSession() as session:
```
This line creates an asynchronous context manager using the aiohttp.ClientSession() class. This class manages an HTTP session and can be used to make requests. By using the async with statement, the session will be automatically closed when the context is exited.
```python
async with session.get(url, params=params) as response:
```

Inside the aiohttp.ClientSession() context, this line makes an asynchronous GET request to the given URL with the specified query parameters (provided as a dictionary in params). The response is then stored in the response variable. Like the session, the response is also managed by an asynchronous context manager, ensuring that resources are properly released when the context is exited.
```python
return await response.text()
```
This line retrieves the response text asynchronously by calling the response.text() method, which is a coroutine. The await keyword is used to obtain the result of this coroutine, and the resulting text is returned.

This code would typically be used inside an async def function, allowing the function to be executed concurrently with other asynchronous tasks.

### Resources

[Youtube video - async scraping, semaphore ](https://www.youtube.com/watch?v=6ow7xloFy5s&ab_channel=CodingEntrepreneurs)